In [1]:
import pdfreader
from itertools import islice
import re
import tqdm

In [2]:
from pdfreader import PDFDocument, SimplePDFViewer

In [3]:
fd = open('RDTE_Vol1_DARPA_MasterJustificationBook_PB_2022.pdf', 'rb')
doc = PDFDocument(fd)

In [4]:
doc.header.version

'1.4'

In [5]:
doc.metadata

{'Keywords': 'Budget',
 'ModDate': datetime.datetime(2021, 5, 19, 10, 22, 35, tzinfo=tzoffset(None, -14400)),
 'CreationDate': datetime.datetime(2021, 5, 19, 10, 22, 34, tzinfo=tzoffset(None, -14400)),
 'Producer': 'iText 2.1.7 by 1T3XT',
 'Subject': 'Budget Justification',
 'Author': 'Office of the Under Secretary of Defense (Comptroller)',
 'Title': 'Justification Book'}

In [6]:
doc.root.Type

'Catalog'

In [21]:
page_21 = next(islice(doc.pages(), 20, 21))

In [22]:
page_21.MediaBox

[0, 0, 792, 612]

In [23]:
page_six.Parent.Type

'Pages'

In [32]:
page_six.Parent.Parent.Count

100

In [55]:
viewer = SimplePDFViewer(fd)

In [56]:
page_texts = {}
for i, canvas in tqdm.tqdm(enumerate(viewer)):
    page_texts[i] = canvas.text_content

254it [02:09,  1.96it/s]


In [25]:
len(page_texts)

254

In [95]:
def extract_content(page_text):
    content = re.findall('\[\(.*\)\]', page_text)
    return content[27:]

In [20]:
page_texts[21]

'\n q\n q\n1 0 0 -1 0 612 cm\n q\n1 0 0 1 18 54 cm\n0 g\n BT\n/F16 12 Tf\n1 0 0 -1 333.70800781 10.30799961 Tm\n[(UNCLASSIFIED)] TJ\n ET\n Q\n q\n1 0 0 1 18 54 cm\n q\n1 0 0 1 0.25 17.75 cm\n q\n-0.25 0.25 m\n-0.5 0 l\n756 0 l\n755.75 0.25 l\n755.5 0.5 l\n0 0.5 l\n h\n W\n n\n0 G\n[] 0 d\n0.5 w\n-0.5 0.25 m\n756 0.25 l\n S\n Q\n q\n755.75 0.25 m\n756 0 l\n756 487 l\n755.75 486.75 l\n755.5 486.5 l\n755.5 0.5 l\n h\n W\n n\n0 G\n[] 0 d\n0.5 w\n755.75 0 m\n755.75 487 l\n S\n Q\n q\n755.75 486.75 m\n756 487 l\n-0.5 487 l\n-0.25 486.75 l\n0 486.5 l\n755.5 486.5 l\n h\n W\n n\n0 G\n[] 0 d\n0.5 w\n-0.5 486.75 m\n756 486.75 l\n S\n Q\n q\n-0.25 486.75 m\n-0.5 487 l\n-0.5 0 l\n-0.25 0.25 l\n0 0.5 l\n0 486.5 l\n h\n W\n n\n0 G\n[] 0 d\n0.5 w\n-0.25 0 m\n-0.25 487 l\n S\n Q\n Q\n Q\n q\n1 0 0 1 18 558 cm\n0 g\n BT\n/F15 10 Tf\n1 0 0 -1 0 18.19499969 Tm\n[(PE 0601101E: )] TJ\n/F17 10 Tf\n1 0 0 -1 67.20999908 18.19499969 Tm\n[(DEFENSE RESEARCH SCIENCES)] TJ\n ET\n q\n1 0 0 1 324 0 cm\n BT\n/F16 12 

In [49]:
content_21 = extract_content(page_texts[20])
for i, text in tqdm.tqdm(enumerate(content_21)):
    if "DEFENSE RESEARCH SCIENCES" in text[2:-2]:
        print('preface', i)
    if 'A. Mission Description' in text[2:-2]:
        print('A', i) 
    if 'B. Program Change Summary ($ in Millions)' in text[2:-2]:
        print('B', i)

preface 2
preface 26
A 133


In [96]:
master_content = []
for i in tqdm.tqdm(range(20, 254)):
    master_content += extract_content(page_texts[i])

100%|████████████████████████████████████████████████████| 234/234 [00:00<00:00, 20658.55it/s]


In [97]:
def flag_title(content):
    search_object = re.search('\[\([A-Z]\. (.*)', content)
    return search_object

In [104]:
benchmarks = []
for i, content in enumerate(master_content):
    match = flag_title(content)
    if match:
        benchmarks.append(i)

In [105]:
benchmarks[0], benchmarks[1]

(106, 141)

In [108]:
splits = [master_content[i:j] for i,j in zip(benchmarks, benchmarks[1:]+[None])]

In [128]:
def flag_project_title(content):
    search_object = re.search(r'(\[\(T\) 18 \(itle: \))', content)
    return search_object
def flag_fy(content):
    search_object = re.search(r'(\[\(FY [0-9]{4})', content)
    return search_object

In [143]:
benchmarks = []
flip = True
for i, content in enumerate(master_content):
    match = None
    if flip:
        match = flag_project_title(content)
    elif not flip:
        match = flag_fy(content)
    if match:
        print(i, content)
        benchmarks.append(i)
        flip = not flip

268 [(T) 18 (itle: )]
283 [(FY 2021 Plans:)]
340 [(T) 18 (itle: )]
355 [(FY 2021 Plans:)]
386 [(T) 18 (itle: )]
399 [(FY 2021 Plans:)]
431 [(T) 18 (itle: )]
442 [(FY 2021 Plans:)]
466 [(T) 18 (itle: )]
479 [(FY 2021 Plans:)]
496 [(T) 18 (itle: )]
509 [(FY 2020)]
554 [(T) 18 (itle: )]
567 [(FY 2021 Plans:)]
593 [(T) 18 (itle: )]
603 [(FY 2021 Plans:)]
621 [(T) 18 (itle: )]
635 [(FY 2020)]
658 [(T) 18 (itle: )]
669 [(FY 2021 Plans:)]
691 [(T) 18 (itle: )]
705 [(FY 2021 Plans:)]
735 [(T) 18 (itle: )]
748 [(FY 2021 Plans:)]
771 [(T) 18 (itle: )]
788 [(FY 2021 Plans:)]
814 [(T) 18 (itle: )]
826 [(FY 2022 Plans:)]
839 [(T) 18 (itle: )]
848 [(FY 2020)]
870 [(T) 18 (itle: )]
892 [(FY 2020)]
902 [(T) 18 (itle: )]
913 [(FY 2021 Plans:)]
920 [(T) 18 (itle: )]
929 [(FY 2021 Plans:)]
1023 [(T) 18 (itle: )]
1035 [(FY 2021 Plans:)]
1054 [(T) 18 (itle: )]
1067 [(FY 2021 Plans:)]
1079 [(T) 18 (itle: )]
1093 [(FY 2020)]
1109 [(T) 18 (itle: )]
1124 [(FY 2021 Plans:)]
1137 [(T) 18 (itle: )]
1162 [(FY 2020

In [151]:
len(splits)/2

234.0

In [152]:
len(benchmarks[::2]), len(benchmarks[1::2])

(234, 234)

In [154]:
splits = [master_content[i:j] for i,j in zip(benchmarks[::2], benchmarks[1::2])]
splits[4]

['[(T) 18 (itle: )]',
 '[(Young Faculty Award \\(YFA\\))]',
 '[(Description: )]',
 '[(The goal of the Young Faculty Award \\(YFA\\) program is to encourage junior faculty at universities and their)]',
 '[(equivalent at non-profit science and technology research institutions to participate in sponsored research programs that will)]',
 '[(augment capabilities for future defense systems. This program focuses on cutting-edge technologies for greatly enhancing)]',
 '[(microsystems technologies, biological technologies, and defense sciences. The long-term goal for this program is to develop the)]',
 '[(next generation of scientists, engineers, and mathematicians in key disciplines who will focus a significant portion of their careers)]',
 '[(on DoD and national security issues. The aim is for YFA recipients to receive deep interactions with DARPA program managers,)]',
 '[(programs, performers, and the user community. Current activities include research in fifteen topic areas spanning from Ma

In [155]:
import pickle
with open('dod_projects.pickle', 'wb') as f:
    pickle.dump(splits, f)

In [ ]:
###### content_22 = extract_content(page_texts[21])
for i, text in enumerate(content_22):
    if "DEFENSE RESEARCH SCIENCES" in text[2:-2]:
        print('preface', i)
    if 'A. Mission Description' in text[2:-2]:
        print('A', i) 
    if 'B. Program Change Summary' in text[2:-2]:
        print('B', i)

In [48]:
content_22[27:44]

['[(and magnetic components to reduce the size of Electromagnetic \\(EM\\) and sensing systems. Other research could support field-portable electronics with reduced power)]',
 '[(requirements, ultra-high density information storage "on-a-chip", and new approaches to nanometer-scale structures, molecules, and devices.)]',
 '[(The Beyond Scaling Sciences project supports investigations into materials, devices, and architectures to provide continued improvements in electronics performance)]',
 "[(with or without the benefit of Moore's Law \\(silicon scaling\\). Within the next ten years, traditional scaling will start to encounter the fundamental physical limits of silicon,)]",
 '[(requiring fresh approaches to new electronic systems. Over the short term, DoD will therefore need to unleash circuit specialization in order to maximize the benefit of)]',
 '[(traditional silicon. Over the longer term, DoD and the nation will need to engage the computer, material, and mechanical sciences to ex

In [42]:
page_21_text = page_21_canvas.text_content.encode('ascii', 'ignore').decode()

In [55]:
match = re.search('\[\(A. Mission Description', page_21_text)
match.start()

66434

In [62]:
re.findall('\[\(.*\)\]', page_21_text[match.start():])

['[(A. Mission Description and Budget Item Justification)]',
 '[(The Defense Research Sciences Program Element is budgeted in the Basic Research Budget Activity because it provides the technical foundation for long-term National)]',
 '[(Security enhancement through the discovery of new phenomena and the exploration of the potential of such phenomena for Defense applications.  It supports the)]',
 '[(scientific study and experimentation that is the basis for more advanced knowledge and understanding in information, electronic, mathematical, computer, and materials)]',
 '[(sciences. This Program Element also supports innovation and robust transition planning in the technology cycle by working with entrepreneurs to increase the likelihood)]',
 '[(that DARPA funded technologies take root in the U.S. and provide new capabilities for national defense.)]',
 '[(The Math and Computer Sciences project supports scientific study and experimentation on new mathematical and computational algorithms,

"[(A. Mission Description and Budget Item Justification)] TJ\n ET\n0 11.509 250.320007 0.262 re\n f\n Q\n q\n1 0 0 1 5 245.97 cm\n0 g\n BT\n/F15 10 Tf\n1 0 0 -1 0 8.59500027 Tm\n[(The Defense Research Sciences Program Element is budgeted in the Basic Research Budget Activity because it provides the technical foundation for long-term National)] TJ\n1 0 0 -1 0 20.59499931 Tm\n[(Security enhancement through the discovery of new phenomena and the exploration of the potential of such phenomena for Defense applications.  It supports the)] TJ\n1 0 0 -1 0 32.59500122 Tm\n[(scientific study and experimentation that is the basis for more advanced knowledge and understanding in information, electronic, mathematical, computer, and materials)] TJ\n1 0 0 -1 0 44.59500122 Tm\n[(sciences. This Program Element also supports innovation and robust transition planning in the technology cycle by working with entrepreneurs to increase the likelihood)] TJ\n1 0 0 -1 0 56.59500122 Tm\n[(that DARPA funded techn